In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
schema = 'silver'
table= 'breweries'
path_source = 'abfss://bronze@dlsbreweries.dfs.core.windows.net/*.parquet'
path_target = 'abfss://silver@dlsbreweries.dfs.core.windows.net/breweries'

In [0]:
df = spark.read.format("parquet").option("header", True).load(path_source)

In [0]:
df.createOrReplaceTempView("bronze_breweries")

In [0]:
df = spark.sql(
f"""
SELECT
    name,
    brewery_type,
    CASE 
        WHEN address_1 = 'Unnamed Street' OR address_1 IS NULL THEN address_2
        ELSE address_1
    END AS address,
    city,
    state,
    country
FROM bronze_breweries
""")

In [0]:
df = df.withColumn('load_timestamp', current_timestamp())

In [0]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema}")

DataFrame[]

In [0]:
df.write \
    .format('delta') \
    .mode('overwrite') \
    .partitionBy('state') \
    .option('path', path_target) \
    .option('overwriteSchema', True) \
    .saveAsTable(f'{schema}.{table}')

Data saved successfully.


In [0]:
spark.sql(f'OPTIMIZE {schema}.{table}')

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,